In [ ]:
# ============================
# IMPORTANDO LO NECESARIO
# ============================
import folium
from folium.plugins import MarkerCluster
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# ============================
# DATOS
# ============================
# Cargar distritos (GeoJSON con geometrías por distrito) (relative to repo root)
districts = gpd.read_file("data/peru_distrital_simple.geojson") # <-- Geojson para folium: distritos de perú


# DataFrame hospitales
url = https://docs.google.com/spreadsheets/d/<SHEET_ID>/export?format=csv&gid=<GID>](https://docs.google.com/spreadsheets/d/1xOkeqlTCWVifWmfcVlvo2geTjDvoTAS8DIvkaCW9u64/export?format=csv&gid=287328050
hospitals_df = pd.read_csv(url)


In [ ]:
# ============================
# Task 1: Choropleth hospitales por distrito
# ============================
# Acomodamos las variables y creamos variable de conteo hospitales por distrito
hospitals_per_district = hospitals_df.groupby("DISTRITO").size().reset_index(name="count")
districts = districts.rename({'NOMBDIST':'DISTRITO'}, axis = 1 ) # Renombramos la columna para que coincida con el df de hospitales
districts = districts.rename({'NOMBDEP':'DEPARTAMENTO'}, axis = 1 ) # Renombramos la columna para que coincida con el df de hospitales
districts = districts.merge(hospitals_per_district, how="inner" on="DISTRITO", right_on="district", how="left")

# ============================
# MAPA BASE
# ============================
zoom_start = 10
lat_hospital = hospitals_df["LATITUD"].mean()
long_hospital = hospitals_df["LONGITUD"].mean()
a = fm.Map(location = [lat_hospital, long_hospital], tiles ="Cartodb Poistron", zoom_start = zoom_start, control_scale=True)

fm.Choropleth(
    geo_data=districts,
    data=districts,
    columns=["district_name", "count"],
    key_on="feature.properties.district_name",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Número de hospitales por distrito"
).add_to(a)

# Agregar cluster de hospitales
marker_cluster = MarkerCluster().add_to(a)
for _, row in hospitals_df.iterrows():
    folium.Marker(
        location=[row["LATITUD"], row["LONGITUD"]],
        popup=f"Hospital en distrito: {row['DISTRITO']}"
    ).add_to(marker_cluster)


In [ ]:
# ============================
# Task 2: Proximidad Lima & Loreto
# ============================

# Crear geometría
hospitals_gdf = gpd.GeoDataFrame(
    hospitals_df,
    geometry=gpd.points_from_xy(hospitals_df.LONGITUD, hospitals_df.LATITUD),
    crs="EPSG:4326"

# ============================
# 1. Reproyectar ambos a métrico (EPSG:3857)
# ============================
hospitals_gdf = hospitals_gdf.to_crs(epsg=3857)
districts = districts.to_crs(epsg=3857)

# ============================
# 2. Crear buffer de 10 km y contar hospitales dentro
# ============================
districts["buffer_10km"] = districts.buffer(10000)

hospital_counts = []
for idx, row in districts.iterrows():
    buffer = row["buffer_10km"]
    count = hospitals_gdf.within(buffer).sum()
    hospital_counts.append(count)

districts["hospital_density"] = hospital_counts

# ============================
# 3. Análisis de proximidad
# ============================

# Crear mapa base
zoom_start = 10
lat_hospital = hospitals_df["LATITUD"].mean()
long_hospital = hospitals_df["LONGITUD"].mean()
b = fm.Map(location = [lat_hospital, long_hospital], tiles ="Cartodb Poistron", zoom_start = zoom_start, control_scale=True)

# Iterar por cada departamento (Lima y Loreto)
for departamento in ["LIMA", "LORETO"]:
    departamento = districts[districts["DEPARTAMENTO"] == DEPARTAMENTO]

    # Mayor densidad
    high = departamento.loc[departamento["hospital_density"].idxmax()]
    folium.Circle(
        location=[high.geometry.y, high.geometry.x],
        radius=10000,
        color="green",
        fill=True,
        popup=f"{high['name']} ({departamento}) - Alta densidad: {high['hospital_density']}"
    ).add_to(b)

    # Menor densidad
    low = departamento.loc[departamento["hospital_density"].idxmin()]
    folium.Circle(
        location=[low.geometry.y, low.geometry.x],
        radius=10000,
        color="red",
        fill=True,
        popup=f"{low['name']} ({departamento}) - Baja densidad: {low['hospital_density']}"
    ).add_to(b)

# Guardar
b.save("mapa_proximidad.html")


# ============================
# 3. Análisis corto
# ============================
# Lima:
# La concentración urbana de los hospitales evidencia una centralización de los servicios hospitalarios.
# En un radio de 10 km, la accesibilidad es alta y muestra mayor facilidad para acceder a estos servicios a comparación de Loreto.

# Loreto:
# Hay una baja densidad hospitalaria dentro de los radios de 10km, lo que muestra una amplia brecha en accesbilidad para estos servicios.
# Esto podría originarse debido a la dispersión geográfica en la Amazonía y las dificultades que representa el transportarse en esta región.
